In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict



In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
from sql_init import Review, Base
 
engine = create_engine('sqlite:///reviews.db')
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [3]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.implicitly_wait(10)


In [4]:
driver.get("https://www.amazon.com/s/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aadidas%2Cp_89%3Aadidas&keywords=adidas&ie=UTF8&qid=1507481929&rnid=2528832011")

In [ ]:
links = []
elements = []
for i in driver.find_elements_by_class_name("s-access-title"):
    links.append(i.find_element_by_xpath('..').get_attribute('href'))
    elements.append(i)

In [ ]:
elements[0].click()

In [ ]:
a_s = driver.find_elements_by_tag_name('a')
#TODO Error handling for when there are no reviews
for e in a_s:
    if re.search('See all [0-9,.]* customer reviews', e.text):
        e.click()
        break

In [ ]:
e = driver.find_element_by_class_name('a-last')

In [ ]:
e = e.find_element_by_tag_name('a')

In [ ]:
e.text

In [ ]:
e.click()

In [ ]:
reviews = defaultdict(dict)
for e in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
    star_rating = e.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
    e = e.find_element_by_xpath('../../../..')
#     review_id = e.get_attribute('id')
#     reviews[review_id]['star_rating'] = star_rating
#     reviews[review_id]['review_title'] = e.find_element_by_class_name('review-title').text
#     reviews[review_id]['review_date'] = e.find_element_by_class_name('review-date').text[3:]
#     reviews[review_id]['text'] = e.find_element_by_class_name('review-text').text
    
    new_review = Review(review_id = e.get_attribute('id'),
    star_rating = int(star_rating),
    review_title = e.find_element_by_class_name('review-title').text,
    review_date = e.find_element_by_class_name('review-date').text[3:],
    text = e.find_element_by_class_name('review-text').text)
    session.add(new_review)
    session.commit()
    
    
    
    
    
    
    
    

In [ ]:
reviews

In [ ]:
for e in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
    e.parent.click()
    break